In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from functools import reduce

In [2]:
# load data
data_2016 = pd.read_csv('CDS_2016_va.csv')
data_2017 = pd.read_csv('CDS_2017_va.csv')
data_2018 = pd.read_csv('CDS_2018_va.csv')
data_2019 = pd.read_csv('CDS_2019_NO_LABEL.csv')
asignaturas = pd.read_csv('Asignaturas.csv', sep = ';')
clientes = pd.read_csv('Clientes.csv', sep = ';')
cursos = pd.read_csv('Cursos.csv', sep = ';')
lengua = pd.read_csv('Lengua.csv', sep = ';')
tme = pd.read_csv('TME.csv', sep = ';')
ts = pd.read_csv('TS.csv', sep = ';')

In [3]:
# entries in column 'Curso' starts with a 'c' for data_2018
data_2018['Curso'] =  data_2018.Curso.str[1:].astype(int).copy()

# entries in column 'Año natural' are 18 instead of 2018
data_2018['Año natural'] = 2018

# some column labels for data_2016 are different
data_2016.columns = data_2017.columns

# data_2017 has duplicate rows
data_2017.drop_duplicates(subset = None, keep = 'first', inplace = True)

In [4]:
### Asignatura ###

# remove non-numerical values from the asignatura code
asignaturas = asignaturas.loc[~asignaturas.Asignatura.isin(['EE01', '#']),:].copy()

# convert the asignatura code the integer
asignaturas['Asignatura'] = asignaturas.Asignatura.astype(int).copy()

### Curso ###

# remove non-numerical values from the curso code
cursos = cursos.loc[~cursos.Curso.isin(['#']),:].copy()

# convert the curso code the integer
cursos['Curso'] = cursos.Curso.astype(int).copy()

# keep only relevant columns
cursos = cursos[['Curso', 'N_Curso']].copy()

### TME ###

# remove non-numerical values from the curso code
tme = tme.loc[~tme['Tipo Material Educat'].isin(['#']),:].copy()

# convert the curso code the integer
tme['Tipo Material Educativo'] = tme['Tipo Material Educat'].astype(int).copy()

# drop original column
tme.drop('Tipo Material Educat', axis = 1, inplace = True)

### TS ###

# remove non-numerical values from the curso code
ts = ts.loc[~ts['Tipo Soporte Actual'].isin(['#']),:].copy()

# convert the curso code the integer
ts['Tipo Soporte Actual'] = ts['Tipo Soporte Actual'].astype(int).copy()

In [5]:
# create list with all datasets
datasets = [data_2016, data_2017, data_2018, data_2019]

# loop over this list
for i, data in enumerate(datasets):
    
    # join data
    datasets[i] = datasets[i].merge(asignaturas, how = 'left', on = 'Asignatura')
    datasets[i] = datasets[i].merge(clientes, how = 'left', on = 'Id_Cliente')
    datasets[i] = datasets[i].merge(cursos, how = 'left', on = 'Curso')
    datasets[i] = datasets[i].merge(lengua, how = 'left', on = 'Lengua')
    datasets[i] = datasets[i].merge(tme, how = 'left', on = 'Tipo Material Educativo')
    datasets[i] = datasets[i].merge(ts, how = 'left', on = 'Tipo Soporte Actual')
    
    # create a unique record id
    datasets[i]['record_id'] = list(map(lambda a, b, c, d, e, f: str(a) + '_' + \
                                                                 str(b) + '_' + \
                                                                 str(c) + '_' + \
                                                                 str(d) + '_' + \
                                                                 str(e) + '_' + \
                                                                 str(f),
                                        datasets[i]['Id_Cliente'],
                                        datasets[i]['Curso'],
                                        datasets[i]['Asignatura'],
                                        datasets[i]['Tipo Material Educativo'],
                                        datasets[i]['Lengua'],
                                        datasets[i]['Tipo Soporte Actual']))
    
    # create a column for non use, use exception throwing because data_2019 doesn't have the 'Grupo Editorial' column
    try:
        # create column name in the form non_use_YYYY
        column_name = 'non_use_' + str(datasets[i]['Año natural'].unique()[0])
        
        # create binary column with 1 where Grupo Editorial = 90 (non-use)
        datasets[i][column_name] = np.where(datasets[i]['Grupo Editorial'] == 90, 1, 0)
        
    except:
        pass

# merge all four dataframes into one on the record_id
target_df = reduce(lambda x, y: pd.merge(x, y, how = 'outer', on = 'record_id'), datasets)

# keep only the non_use columns and the record_id
target_df = target_df[['record_id', 'non_use_2016', 'non_use_2017', 'non_use_2018']].copy()

# create the target 
target_df['target_2017'] = np.where((target_df.non_use_2016 == 0) & (target_df.non_use_2017 == 1), 1, 0)
target_df['target_2018'] = np.where((target_df.non_use_2017 == 0) & (target_df.non_use_2018 == 1), 1, 0)

# add target to dataframes for 2016 and 2017
datasets[1] = datasets[1].merge(target_df[['record_id', 'target_2017']], how = 'left', on = 'record_id')
datasets[2] = datasets[2].merge(target_df[['record_id', 'target_2018']], how = 'left', on = 'record_id')

# rename columns to allow concatenation on these columns
datasets[0] = datasets[0].rename(columns = {'non_use_2016':'non_use'})
datasets[1] = datasets[1].rename(columns = {'non_use_2017':'non_use', 'target_2017':'target'})
datasets[2] = datasets[2].rename(columns = {'non_use_2018':'non_use', 'target_2018':'target'})

# add column indicating if the record is a non_user in the previous year
datasets[1] = datasets[1].merge(datasets[0][['record_id', 'non_use', 'Grupo Editorial']], 
                                how = 'left', 
                                on = 'record_id', 
                                suffixes = ('', '_previous'))

datasets[2] = datasets[2].merge(datasets[1][['record_id', 'non_use', 'Grupo Editorial']], 
                                how = 'left', 
                                on = 'record_id', 
                                suffixes = ('', '_previous'))

datasets[3] = datasets[3].merge(datasets[2][['record_id', 'non_use', 'Grupo Editorial']], 
                                how = 'left', 
                                on = 'record_id', 
                                suffixes = ('', '_previous'))

# concatenate data into one dataframe
data = pd.concat(datasets, sort = False, ignore_index = True)

# rename columns
data.rename(columns = {'Id_Cliente' : 'client_id', 
                       'Año natural' : 'year', 
                       'Curso' : 'course_code',
                       'Asignatura' : 'subject_code',
                       'Tipo Material Educativo' : 'material_type_code', 
                       'Grupo Editorial' : 'editorial_group', 
                       'Lengua' : 'language_code',
                       'Tipo Soporte Actual' : 'media_support_code',
                       'Variable1' : 'class_size',
                       'Variable2' : 'monetary_value',
                       'N_Asignatura' : 'subject_name',
                       'Latitud' : 'latitude', 
                       'Longitud' : 'longitude', 
                       'Comunidad Autónoma' : 'state', 
                       'Id_Asociación' : 'association_id',
                       'Id_Subasociación' : 'subassociation_id', 
                       'Titularidad' : 'school_type', 
                       'N_Curso' : 'course_name', 
                       'N_Lengua' : 'language_name', 
                       'N_TME' : 'material_type_name',
                       'N_TS' : 'media_support_name',
                       'Grupo Editorial_previous' : 'editorial_group_previous'},
            inplace = True)

# delete unnecessary variables to free up RAM
del asignaturas, clientes, cursos, lengua, tme, ts, data_2016, data_2017, data_2018, data_2019, datasets, target_df